# Machine Learning Training Datasheet

In [3]:
# importing the libraries

from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, transpile, Aer, ClassicalRegister
from qiskit.quantum_info import state_fidelity
from qiskit.tools.visualization import plot_histogram
import qiskit.ignis.mitigation.measurement as mc
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import numpy as np
import math as m
import pandas as pd
import cvxopt
import time
import copy

In [15]:
# IBMQ.load_account()
provider=IBMQ.get_provider('ibm-q')
deviceV=provider.get_backend('ibmq_vigo')
# M_simulator = provider.backends('ibmq_qasm_simulator')[0]

In [5]:
#picks function picks up the probability of the first qubit to be state 'k'
def picks(g,n,k):
    # g is the raw data
    # n is the no of qubit you want to omit
    # k is the final state you want from the raw data|
    effective_f={}
    effective={}
    f=copy.deepcopy(g)
    # we don't want the data to be changed, so we make a copy of that
    for i in range(len(f)):
        for key in f[i].keys():
            f[i][key] = f[i][key]/8192
        data=f[i].keys()
        state=[]
        for j in data:
            state.append(j)
        final_state=[u[:n] for u in state]
        effective_state=[]
        for j in range(len(state)):
            if final_state[j] == k :
                effective_state.append(state[j])
                    
        effective[i]={key:f[i][key] for key in effective_state}
        
    return effective

In [94]:
def TD1(C,D):
    td1 = np.sqrt(1-(np.inner(C,D))**2)
    return td1

DF = pd.read_csv( 'D:\\emails.csv' )

In [96]:
DF

,No.,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,E21,E22,E23,E24,E25,E26,E27,E28,E29,E30
0,1,0.984375,0.947021,0.972900,0.978516,0.941162,0.979614,0.927124,0.909790,0.944214,...,0.370483,0.399536,0.418457,0.438843,0.395264,0.391479,0.327026,0.389038,0.647949,0.671631
1,2,0.987671,0.944580,0.974609,0.978027,0.939941,0.978638,0.932129,0.908203,0.937988,...,0.367554,0.402832,0.413086,0.438843,0.394287,0.400024,0.328979,0.389404,0.651123,0.670166
2,3,0.987183,0.943237,0.972656,0.975342,0.944824,0.975220,0.926270,0.911865,0.943359,...,0.363525,0.406250,0.411377,0.429199,0.389648,0.398193,0.329224,0.391846,0.660034,0.674927
3,4,0.987183,0.944214,0.974121,0.981812,0.944824,0.980103,0.926758,0.911621,0.940674,...,0.357056,0.397095,0.415039,0.438599,0.402100,0.408569,0.322021,0.386597,0.648804,0.672852
4,5,0.988037,0.945679,0.975342,0.975464,0.942993,0.976562,0.932739,0.913208,0.940796,...,0.369385,0.404053,0.427002,0.441650,0.399170,0.400146,0.327759,0.388672,0.643188,0.657837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,60,0.986572,0.947510,0.972656,0.977295,0.949585,0.976196,0.930054,0.917603,0.943481,...,0.381104,0.412598,0.427124,0.454956,0.401978,0.426270,0.340332,0.394165,0.638550,0.689087
60,61,0.987671,0.938110,0.975098,0.979492,0.941284,0.978271,0.926025,0.911865,0.939209,...,0.376343,0.402588,0.430664,0.459106,0.399414,0.424561,0.336914,0.401123,0.653442,0.685181
61,62,0.988159,0.941772,0.974731,0.977905,0.938843,0.974243,0.930420,0.911743,0.941772,...,0.383423,0.405273,0.428955,0.449951,0.395020,0.419067,0.338623,0.415283,0.641479,0.683716
62,63,0.987061,0.947998,0.976807,0.976196,0.941650,0.978149,0.925903,0.909790,0.938843,...,0.384033,0.405762,0.422363,0.450928,0.401245,0.420044,0.334961,0.401367,0.651611,0.683716


In [7]:
# #===================================================Email-1--00=======================================================#

# #cir1 is the fist example
# cir1=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir1.append(qc1)

# job1=execute(cir1, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir1:
#     list[j] = job1.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listSpe1 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe1 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe1=[]
# listNspe1=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe1.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe1.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe1.dat', listspe1)
# np.savetxt('listNspe1.dat', listNspe1)

In [10]:
# print(cir1[63])

        ░ ┌────────────────────┐ ░ ┌─┐   
q63_0: ─░─┤ U3(pi/2,pi/2,pi/2) ├─░─┤M├───
        ░ └────────────────────┘ ░ └╥┘┌─┐
q63_1: ─░────────────────────────░──╫─┤M├
        ░                        ░  ║ └╥┘
c63_0: ═════════════════════════════╩══╬═
                                       ║ 
c63_1: ════════════════════════════════╩═
                                         


In [98]:
#===================================================Email-1--00=======================================================#

dataE1=[]
for i in DF["E1"]:
    dataE1.append(i)

    TDE1 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE1:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE1.append(tdi)
np.savetxt('TrDist_E1.dat', TDE1)

In [99]:
# #===================================================Email-2--00+01=======================================================#

# #cir2 is the fist example
# cir2=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)
            
#             qc1.h(0)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir2.append(qc1)

# job2=execute(cir2, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir2:
#     list[j] = job2.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe2 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe2 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe2=[]
# listNspe2=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe2.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe2.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe2.dat', listspe2)
# np.savetxt('listNspe2.dat', listNspe2)

In [100]:
# print(cir2[63])

In [101]:
#===================================================Email-2--00+01=======================================================#

dataE2=[]
for i in DF["E2"]:
    dataE2.append(i)

    TDE2 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE2:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE2.append(tdi)
np.savetxt('TrDist_E2.dat', TDE2)

In [102]:
# #===================================================Email-3--00=======================================================#

# #cir3 is the fist example
# cir3=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir3.append(qc1)

# job3=execute(cir3, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir3:
#     list[j] = job3.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe3 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe3 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe3=[]
# listNspe3=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe3.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe3.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe3.dat', listspe3)
# np.savetxt('listNspe3.dat', listNspe3)

In [103]:
# print(cir3[63])

In [104]:
#===================================================Email-3--00=======================================================#

dataE3=[]
for i in DF["E3"]:
    dataE3.append(i)

    TDE3 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE3:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE3.append(tdi)
np.savetxt('TrDist_E3.dat', TDE3)

In [105]:
# #===================================================Email-4--00=======================================================#

# #cir4 is the fist example
# cir4=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir4.append(qc1)

# job4=execute(cir4, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir4:
#     list[j] = job4.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe4 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe4 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe4=[]
# listNspe4=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe4.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe4.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe4.dat', listspe4)
# np.savetxt('listNspe4.dat', listNspe4)

In [106]:
# print(cir4[63])

In [107]:
#===================================================Email-4--00=======================================================#

dataE4=[]
for i in DF["E4"]:
    dataE4.append(i)

    TDE4 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE4:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE4.append(tdi)
np.savetxt('TrDist_E4.dat', TDE4)

In [108]:
# #===================================================Email-5--00=======================================================#

# #cir5 is the fist example
# cir5=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir5.append(qc1)

# job2=execute(cir5, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir5:
#     list[j] = job2.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe5 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe5 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe5=[]
# listNspe5=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe5.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe5.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe5.dat', listspe5)
# np.savetxt('listNspe5.dat', listNspe5)

In [109]:
# print(cir5[63])

In [110]:
#===================================================Email-5--00=======================================================#

dataE5=[]
for i in DF["E5"]:
    dataE5.append(i)

    TDE5 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE5:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE5.append(tdi)
np.savetxt('TrDist_E5.dat', TDE5)

In [111]:
# #===================================================Email-6--00=======================================================#

# #cir6 is the fist example
# cir6=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir6.append(qc1)

# job6=execute(cir6, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir6:
#     list[j] = job6.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe6 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe6 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe6=[]
# listNspe6=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe6.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe6.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe6.dat', listspe6)
# np.savetxt('listNspe6.dat', listNspe6)

In [112]:
# print(cir6[63])

In [113]:
#===================================================Email-6--00=======================================================#

dataE6=[]
for i in DF["E6"]:
    dataE6.append(i)

    TDE6 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE6:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE6.append(tdi)
np.savetxt('TrDist_E6.dat', TDE6)

In [114]:
# #===================================================Email-7--00=======================================================#

# #cir7 is the fist example
# cir7=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir7.append(qc1)

# job7=execute(cir7, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir7:
#     list[j] = job7.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe7 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe7 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe7=[]
# listNspe7=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe7.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe7.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe7.dat', listspe7)
# np.savetxt('listNspe7.dat', listNspe7)

In [115]:
# print(cir7[63])

In [116]:
#===================================================Email-7--00=======================================================#

dataE7=[]
for i in DF["E7"]:
    dataE7.append(i)

    TDE7 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE7:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE7.append(tdi)
np.savetxt('TrDist_E7.dat', TDE7)

In [117]:
# #===================================================Email-8--00=======================================================#

# #cir8 is the fist example
# cir8=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir8.append(qc1)

# job8=execute(cir8, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir8:
#     list[j] = job8.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #lsitspe8 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #lsitspe8 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# lsitspe8=[]
# lsitNspe8=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         lsitspe8.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         lsitNspe8.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe8.dat', lsitspe8)
# np.savetxt('listNspe8.dat', lsitNspe8)

In [118]:
# print(cir8[63])

In [119]:
#===================================================Email-8--00=======================================================#

dataE8=[]
for i in DF["E8"]:
    dataE8.append(i)

    TDE8 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE8:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE8.append(tdi)
np.savetxt('TrDist_E8.dat', TDE8)

In [120]:
# #===================================================Email-9--01=======================================================#

# #cir9 is the fist example
# cir9=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir9.append(qc1)

# job9=execute(cir9, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir9:
#     list[j] = job9.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe9 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe9 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe9=[]
# listNspe9=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe9.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe9.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe9.dat', listspe9)
# np.savetxt('listNspe9.dat', listNspe9)

In [121]:
# print(cir9[63])

In [122]:
#===================================================Email-9--01=======================================================#

dataE9=[]
for i in DF["E9"]:
    dataE9.append(i)

    TDE9 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE9:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE9.append(tdi)
np.savetxt('TrDist_E9.dat', TDE9)

In [123]:
# #===================================================Email-10--01=======================================================#

# #cir10 is the fist example
# cir10=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir10.append(qc1)

# job10=execute(cir10, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir10:
#     list[j] = job10.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe10 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe10 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe10=[]
# listNspe10=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe10.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe10.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe10.dat', listspe10)
# np.savetxt('listNspe10.dat', listNspe10)

In [124]:
# print(cir10[63])

In [125]:
#===================================================Email-10--01=======================================================#

dataE10=[]
for i in DF["E10"]:
    dataE10.append(i)

    TDE10 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE10:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE10.append(tdi)
np.savetxt('TrDist_E10.dat', TDE10)

In [126]:
# #===================================================Email-11--01=======================================================#

# #cir11 is the fist example
# cir11=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir11.append(qc1)

# job11=execute(cir11, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir11:
#     list[j] = job11.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe11 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe11 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe11=[]
# listNspe11=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe11.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe11.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe11.dat', listspe11)
# np.savetxt('listNspe11.dat', listNspe11)

In [127]:
# print(cir11[63])

In [128]:
#===================================================Email-11--01=======================================================#

dataE11=[]
for i in DF["E11"]:
    dataE11.append(i)

    TDE11 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE11:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE11.append(tdi)
np.savetxt('TrDist_E11.dat', TDE11)

In [129]:
# #===================================================Email-12--00+01=======================================================#

# #cir12 is the fist example
# cir12=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)
            
#             qc1.h(0)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir12.append(qc1)

# job12=execute(cir12, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir12:
#     list[j] = job12.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe12 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe12 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe12=[]
# listNspe12=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe12.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe12.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe12.dat', listspe12)
# np.savetxt('listNspe12.dat', listNspe12)

In [130]:
# print(cir12[63])

In [131]:
#===================================================Email-12--00+01=======================================================#

dataE12=[]
for i in DF["E12"]:
    dataE12.append(i)

    TDE12 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE12:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE12.append(tdi)
np.savetxt('TrDist_E12.dat', TDE12)

In [132]:
# #===================================================Email-13--01=======================================================#

# #cir13 is the fist example
# cir13=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir13.append(qc1)

# job13=execute(cir13, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir13:
#     list[j] = job13.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe13 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe13 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe13=[]
# listNspe13=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe13.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe13.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe13.dat', listspe13)
# np.savetxt('listNspe13.dat', listNspe13)

In [133]:
# print(cir13[63])

In [134]:
#===================================================Email-13--01=======================================================#

dataE13=[]
for i in DF["E13"]:
    dataE13.append(i)

    TDE13 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE13:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE13.append(tdi)
np.savetxt('TrDist_E13.dat', TDE13)

In [135]:
# #===================================================Email-14--01=======================================================#

# #cir14 is the fist example
# cir14=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir14.append(qc1)

# job14=execute(cir14, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir14:
#     list[j] = job14.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe14 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe14 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe14=[]
# listNspe14=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe14.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe14.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe14.dat', listspe14)
# np.savetxt('listNspe14.dat', listNspe14)

In [136]:
# print(cir14[63])

In [137]:
#===================================================Email-14--01=======================================================#

dataE14=[]
for i in DF["E14"]:
    dataE14.append(i)

    TDE14 =[]
#if example is spam (Email No. 1-14 )
A = [1,0]
#if example is not spam (Email No. 15-30)
# A=[0,1]
for i in dataE14:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE14.append(tdi)
np.savetxt('TrDist_E14.dat', TDE14)

In [138]:
# #===================================================Email-15--11=======================================================#

# #cir15 is the fist example
# cir15=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir15.append(qc1)

# job15=execute(cir15, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir15:
#     list[j] = job15.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe15 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe15 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe15=[]
# listNspe15=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe15.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe15.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe15.dat', listspe15)
# np.savetxt('listNspe15.dat', listNspe15)

In [139]:
# print(cir15[63])

In [187]:
#===================================================Email-15--11=======================================================#

dataE15=[]
for i in DF["E15"]:
    dataE15.append(i)

    TDE15 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE15:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE15.append(tdi)
np.savetxt('TrDist_E15.dat', TDE15)

In [141]:
# #===================================================Email-16--11=======================================================#

# #cir16 is the fist example
# cir16=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir16.append(qc1)

# job16=execute(cir16, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir16:
#     list[j] = job16.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe16 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe16 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe16=[]
# listNspe16=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe16.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe16.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe16.dat', listspe16)
# np.savetxt('listNspe16.dat', listNspe16)

In [142]:
# print(cir16[63])

In [143]:
#===================================================Email-16--11=======================================================#

dataE16=[]
for i in DF["E16"]:
    dataE16.append(i)

    TDE16 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE16:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE16.append(tdi)
np.savetxt('TrDist_E16.dat', TDE16)

In [144]:
# #===================================================Email-17--10=======================================================#

# #cir17 is the fist example
# cir17=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir17.append(qc1)

# job17=execute(cir17, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir17:
#     list[j] = job17.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe17 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe17 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe17=[]
# listNspe17=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe17.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe17.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe17.dat', listspe17)
# np.savetxt('listNspe17.dat', listNspe17)

In [145]:
# print(cir17[63])

In [146]:
#===================================================Email-17--10=======================================================#

dataE17=[]
for i in DF["E17"]:
    dataE17.append(i)

    TDE17 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE17:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE17.append(tdi)
np.savetxt('TrDist_E17.dat', TDE17)

In [147]:
# #===================================================Email-18--10=======================================================#

# #cir18 is the fist example
# cir18=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir18.append(qc1)

# job18=execute(cir18, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir18:
#     list[j] = job18.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe18 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe18 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe18=[]
# listNspe18=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe18.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe18.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe18.dat', listspe18)
# np.savetxt('listNspe18.dat', listNspe18)

In [148]:
# print(cir18[63])

In [149]:
#===================================================Email-18--10=======================================================#

dataE18=[]
for i in DF["E18"]:
    dataE18.append(i)

    TDE18 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE18:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE18.append(tdi)
np.savetxt('TrDist_E18.dat', TDE18)

In [150]:
# #===================================================Email-19--10=======================================================#

# #cir19 is the fist example
# cir19=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir19.append(qc1)

# job19=execute(cir19, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir19:
#     list[j] = job19.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe19 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe19 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe19=[]
# listNspe19=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe19.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe19.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe19.dat', listspe19)
# np.savetxt('listNspe19.dat', listNspe19)

In [151]:
# print(cir19[63])

In [152]:
#===================================================Email-19--10=======================================================#

dataE19=[]
for i in DF["E19"]:
    dataE19.append(i)

    TDE19 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE19:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE19.append(tdi)
np.savetxt('TrDist_E19.dat', TDE19)

In [153]:
# #===================================================Email-20--10=======================================================#

# #cir20 is the fist example
# cir20=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir20.append(qc1)

# job20=execute(cir20, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir20:
#     list[j] = job20.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe20 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe20 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe20=[]
# listNspe20=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe20.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe20.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe20.dat', listspe20)
# np.savetxt('listNspe20.dat', listNspe20)

In [154]:
# print(cir20[63])

In [155]:
#===================================================Email-20--10=======================================================#

dataE20=[]
for i in DF["E20"]:
    dataE20.append(i)

    TDE20 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE20:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE20.append(tdi)
np.savetxt('TrDist_E20.dat', TDE20)

In [156]:
# #===================================================Email-21--10=======================================================#

# #cir21 is the fist example
# cir21=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir21.append(qc1)

# job21=execute(cir21, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir21:
#     list[j] = job21.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe21 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe21 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe21=[]
# listNspe21=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe21.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe21.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe21.dat', listspe21)
# np.savetxt('listNspe21.dat', listNspe21)

In [157]:
# print(cir21[63])

In [158]:
#===================================================Email-21--10=======================================================#

dataE21=[]
for i in DF["E21"]:
    dataE21.append(i)

    TDE21 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE21:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE21.append(tdi)
np.savetxt('TrDist_E21.dat', TDE21)

In [159]:
# #===================================================Email-22--10+11=======================================================#

# #cir22 is the fist example
# cir22=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.h(0)
#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir22.append(qc1)

# job22=execute(cir22, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir22:
#     list[j] = job22.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe22 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe22 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe22=[]
# listNspe22=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe22.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe22.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe22.dat', listspe22)
# np.savetxt('listNspe22.dat', listNspe22)

In [160]:
# print(cir22[63])

In [161]:
#===================================================Email-22--10+11=======================================================#

dataE22=[]
for i in DF["E22"]:
    dataE22.append(i)

    TDE22 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE22:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE22.append(tdi)
np.savetxt('TrDist_E22.dat', TDE22)

In [162]:
# #===================================================Email-23--10=======================================================#

# #cir23 is the fist example
# cir23=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir23.append(qc1)

# job23=execute(cir23, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir23:
#     list[j] = job23.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe23 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe23 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe23=[]
# listNspe23=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe23.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe23.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe23.dat', listspe23)
# np.savetxt('listNspe23.dat', listNspe23)

In [163]:
# print(cir23[63])

In [164]:
#===================================================Email-23--10=======================================================#

dataE23=[]
for i in DF["E23"]:
    dataE23.append(i)

    TDE23 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE23:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE23.append(tdi)
np.savetxt('TrDist_E23.dat', TDE23)

In [165]:
# #===================================================Email-24--10=======================================================#

# #cir24 is the fist example
# cir24=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir24.append(qc1)

# job24=execute(cir24, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir24:
#     list[j] = job24.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe24 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe24 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe24=[]
# listNspe24=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe24.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe24.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe24.dat', listspe24)
# np.savetxt('listNspe24.dat', listNspe24)

In [166]:
# print(cir24[63])

In [167]:
#===================================================Email-24--10=======================================================#

dataE24=[]
for i in DF["E24"]:
    dataE24.append(i)

    TDE24 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE24:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE24.append(tdi)
np.savetxt('TrDist_E24.dat', TDE24)

In [168]:
# #===================================================Email-25--10=======================================================#

# #cir25 is the fist example
# cir25=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir25.append(qc1)

# job25=execute(cir25, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir25:
#     list[j] = job25.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe25 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe25 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe25=[]
# listNspe25=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe25.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe25.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe25.dat', listspe25)
# np.savetxt('listNspe25.dat', listNspe25)

In [169]:
# print(cir25[63])

In [170]:
#===================================================Email-25--10=======================================================#

dataE25=[]
for i in DF["E25"]:
    dataE25.append(i)

    TDE25 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE25:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE25.append(tdi)
np.savetxt('TrDist_E25.dat', TDE25)

In [171]:
# #===================================================Email-26--11=======================================================#

# #cir26 is the fist example
# cir26=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir26.append(qc1)

# job26=execute(cir26, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir26:
#     list[j] = job26.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe26 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe26 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe26=[]
# listNspe26=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe26.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe26.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe26.dat', listspe26)
# np.savetxt('listNspe26.dat', listNspe26)

In [172]:
# print(cir26[63])

In [173]:
#===================================================Email-26--11=======================================================#

dataE26=[]
for i in DF["E26"]:
    dataE26.append(i)

    TDE26 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE26:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE26.append(tdi)
np.savetxt('TrDist_E26.dat', TDE26)

In [174]:
# #===================================================Email-27--11=======================================================#

# #cir27 is the fist example
# cir27=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir27.append(qc1)

# job27=execute(cir27, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir27:
#     list[j] = job27.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe27 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe27 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe27=[]
# listNspe27=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe27.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe27.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe27.dat', listspe27)
# np.savetxt('listNspe27.dat', listNspe27)

In [175]:
# print(cir27[63])

In [176]:
#===================================================Email-27--11=======================================================#

dataE27=[]
for i in DF["E27"]:
    dataE27.append(i)

    TDE27 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE27:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE27.append(tdi)
np.savetxt('TrDist_E27.dat', TDE27)

In [177]:
# #===================================================Email-28--11=======================================================#

# #cir28 is the fist example
# cir28=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir28.append(qc1)

# jb28=execute(cir28, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir28:
#     list[j] = jb28.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe28 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe28 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe28=[]
# listNspe28=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe28.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe28.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe28.dat', listspe28)
# np.savetxt('listNspe28.dat', listNspe28)

In [178]:
# print(cir28[63])

In [179]:
#===================================================Email-28--11=======================================================#

dataE28=[]
for i in DF["E28"]:
    dataE28.append(i)

    TDE28 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE28:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE28.append(tdi)
np.savetxt('TrDist_E28.dat', TDE28)

In [180]:
# #===================================================Email-29--11=======================================================#

# #cir29 is the fist example
# cir29=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.x(0)
#             qc1.x(1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir29.append(qc1)

# job29=execute(cir29, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir29:
#     list[j] = job29.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe29 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe29 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe29=[]
# listNspe29=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe29.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe29.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe29.dat', listspe29)
# np.savetxt('listNspe29.dat', listNspe29)

In [181]:
# print(cir29[63])

In [182]:
#===================================================Email-29--11=======================================================#

dataE29=[]
for i in DF["E29"]:
    dataE29.append(i)

    TDE29 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE29:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE29.append(tdi)
np.savetxt('TrDist_E29.dat', TDE29)

In [183]:
# #===================================================Email-30--00+01=======================================================#

# #cir30 is the fist example
# cir30=[]
# for t1 in range(0,4):
#     for p1 in range(0,4):
#         for l1 in range(0,4):
#             q1 = QuantumRegister(2)
#             c1 = ClassicalRegister(2)
#             qc1= QuantumCircuit(q1,c1)

#             qc1.h(0)
#             qc1.cx(0,1)
#             qc1.barrier()
#             qc1.u3((m.pi*t1)/6,(m.pi*p1)/6,(m.pi*l1)/6,q1[0])
#            #qc1.u3(m.pi/(2*t1),m.pi/(2*p1),m.pi/(2*l1),q1[1])
#             qc1.barrier(q1)
#             qc1.measure(q1,c1)
#             cir30.append(qc1)

# jjob30=execute(cir30, backend=deviceV, shots=8192)

# j=0
# list= {}
# for qc1 in  cir30:
#     list[j] = jjob30.result().get_counts(qc1)
#     j=j+1

# list1 = {}
# list1 = picks(list,1,'0')     
# list2 = picks(list,1,'1') 
# #list1 and list2 are python dictionaries with probabilities of the first qubit to be zero; we want a sum over that.

# #listspe30 tells the probability of the example 1 to be spam at different values of theta, lambda and phi
# #listspe30 tells the probability of the example 1 not to be spam at different values of theta, lambda and phi
# listspe30=[]
# listNspe30=[]
# j=0
# for key in list1.keys():
#         for i in list1[key].values():
#                 j=j+i
#         listspe30.append(j)
#         j=0
# for key in list2.keys():
#         for i in list2[key].values():
#                 j=j+i
#         listNspe30.append(j)
#         j=0

# #you will have another 29 lists from the other 29 examples. save them separately, 
# np.savetxt('listspe30.dat', listspe30)
# np.savetxt('listNspe30.dat', listNspe30)

In [184]:
# print(cir30[63])

In [185]:
#===================================================Email-30--00+11=======================================================#

dataE30=[]
for i in DF["E30"]:
    dataE30.append(i)

    TDE30 =[]
#if example is spam (Email No. 1-14 )
# A = [1,0]
#if example is not spam (Email No. 15-30)
A=[0,1]
for i in dataE30:
    B = [np.sqrt(i),np.sqrt(1-i)]
    tdi = TD1(A,B)
    TDE30.append(tdi)
np.savetxt('TrDist_E30.dat', TDE30)